In [76]:
# reporting/quarterly_report/modules/granting.py
from __future__ import annotations

import pandas as pd
import sqlite3
from datetime import datetime
from great_tables import GT, style, loc, html
from pathlib import Path
from reporting.quarterly_report.utils import Database, RenderContext
import numpy as np

# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status, 
    load_report_params                   # to inspect results
)


# ─────────────────────────────────────────────────────────────
# 2) open DB – change path if you work on a copy
# ─────────────────────────────────────────────────────────────
db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")

init_db(db_path=DB_PATH)            # no-op if tables already exist

db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn
report = 'Quarterly_Report'

# ──────────────────────────────────────────────────────────────
# constants – adapt whenever a file-alias changes
# ──────────────────────────────────────────────────────────────

AURI_ALIAS = "audit_result_implementation"
cutoff = pd.to_datetime("2025-04-15")

# ──────────────────────────────────────────────────────────────
# REPORT KIK-OFF
# ──────────────────────────────────────────────────────────────

report_params = load_report_params(report_name=report, db_path=db_path)
table_colors = report_params.get('TABLE_COLORS')


# Define colors if table_colors is not provided
BLUE = table_colors.get("BLUE", "#004A99") if table_colors else "#004A99"
LIGHT_BLUE = table_colors.get("LIGHT_BLUE", "#d6e6f4") if table_colors else "#d6e6f4"
# GRID_CLR = table_colors.get("GRID_CLR", "#004A99") if table_colors else "#004A99"
DARK_BLUE = table_colors.get("DARK_BLUE", "#01244B") if table_colors else "#01244B"
SUB_TOTAL_BACKGROUND = table_colors.get("subtotal_background_color", "#E6E6FA") if table_colors else "#E6E6FA"

# df = fetch_latest_table_data(conn , AURI_ALIAS, cutoff)
# df.columns


In [ ]:
# ──────────────────────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────────────────────
OUTLINE_B = '2px'


def _snapshot(
    df: pd.DataFrame,
    period_end: pd.Timestamp,
    label: str,
    *,
    sources: tuple[str, ...] = ("CAS audits", "ECA audits", "Extensions"),
    amount_filter: int = -200,          # negative adjustments only
) -> pd.DataFrame:
    """
    Return a wide one-period dataframe:

        Indicator | <label> number of cases | <label> in € | <label> % cases
    """
    rows: list[dict[str, object]] = []

    period_df = df[
        (df["AURI_START"] <= period_end) &
        (df["AUDEX_TOTAL_COST_ADJUSTMENT"] <= amount_filter)
    ]

    # ── three ex-post sources ─────────────────────────────────────────
    for src in sources:
        subset = period_df[period_df["AURI_SOURCE"] == src]

        total_cases = len(subset)
        implemented = subset[
            subset["AURI_END_DT"].notna() &
            (subset["AURI_END_DT"] <= period_end)
        ]

        impl_cnt = len(implemented)
        impl_eur = implemented["AUDEX_TOTAL_COST_ADJUSTMENT"].sum()
        pct_impl = (impl_cnt / total_cases * 100) if total_cases else 0

        nice_name = f"{src.split()[0]} participation implemented" \
                    if src != "Extensions" else "Extensions participation implemented"

        rows.append({
            "Indicator": nice_name,
            f"{label} number of cases": impl_cnt,
            f"{label} in €": impl_eur,
            f"{label} % cases": pct_impl,
        })

    return pd.DataFrame(rows)


def determine_epoch_year(cutoff_date: pd.Timestamp) -> int:
    """
    Returns the correct reporting year.
    If the cutoff is in January, then we are reporting for the *previous* year.
    """
    return cutoff_date.year - 1 if cutoff_date.month == 1 else cutoff_date.year


def get_months_in_scope(cutoff: pd.Timestamp) -> list[int]:
    """
    Returns list of month numbers from 1 (January) to the last *full* month before cutoff.
    Handles year rollover if cutoff is in January.
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        end_month = 12
    else:
        year = cutoff.year
        end_month = cutoff.month - 1

    months = pd.date_range(
        start=pd.Timestamp(year=year, month=1, day=1),
        end=pd.Timestamp(year=year, month=end_month, day=1),
        freq="MS"
    ).month.tolist()

    return months


def get_last_date_in_scope(cutoff: pd.Timestamp, months_scope: list[int]) -> pd.Timestamp:
    """
    Returns the last date of the last month in scope.
    For example, if months_scope is [1, 2, 3] (Jan to Mar) and cutoff is in 2025,
    returns 2025-03-31.
    """
    last_month = max(months_scope)
    year = cutoff.year if cutoff.month != 1 else cutoff.year - 1
    # Create a timestamp for the first day of the next month, then subtract 1 day
    next_month_start = pd.Timestamp(year=year, month=last_month + 1, day=1)
    last_date = next_month_start - pd.Timedelta(days=1)
    return last_date


def load_auri_data(db_path: str, cutoff_date: datetime, current_year: int, months_scope: list[int]) -> pd.DataFrame:
    """Load and preprocess AURI data from the database."""

    db = Database(str(DB_PATH))         # thin sqlite3 wrapper
    conn = db.conn
    auri_raw_df = fetch_latest_table_data(conn , "audit_result_implementation", cutoff)

    last_date = get_last_date_in_scope(cutoff_date, months_scope)

    date_cols = ['AURI_START', 'AURI_END_DATE']

    # ▸ strip spaces, then convert
    auri_raw_df[date_cols] = (
        auri_raw_df[date_cols]
        .apply(lambda s: pd.to_datetime(
            s.str.strip(),                 # clean up
            errors='coerce',               # bad → NaT
            format='%Y-%m-%d %H:%M:%S'     # matches 2022-12-19 00:00:00
            ))
        )   

    # filter
    auri_raw_df = auri_raw_df[auri_raw_df['AURI_START'] <= last_date].copy()

    # # set any AURI_END_DATE past cutoff to NaT
    auri_raw_df.loc[
        auri_raw_df['AURI_END_DATE'] > last_date,
        'AURI_END_DATE'
    ] = pd.NaT

    # ───── keep a pure-datetime copy for internal calculations ─┐
    auri_raw_df["AURI_END_DT"] = auri_raw_df["AURI_END_DATE"]     # NEW COLUMN
    # └───────────────────────────────────────────────────────────┘

    # Debug: Check if DataFrame is empty after loading
    if auri_raw_df.empty:
        print("Warning: auri_raw_df is empty after loading from database.")
        return auri_raw_df

    auri_raw_df = auri_raw_df.copy()

    auri_raw_df['TTI_AURI'] = (pd.to_datetime(auri_raw_df['AURI_END_DATE']) - pd.to_datetime(auri_raw_df['AURI_START'])).dt.days
    auri_raw_df['TTI_AURI'] = auri_raw_df['TTI_AURI'].fillna(0).astype(int)
    auri_raw_df['AURI_END_YEAR'] = pd.to_datetime(auri_raw_df['AURI_END_DATE']).dt.year
    auri_raw_df['AURI_END_YEAR'] = auri_raw_df['AURI_END_YEAR'].fillna(0).astype(int)
    auri_raw_df['AURI_END_DATE'] = auri_raw_df['AURI_END_DATE'].fillna('no')
    auri_raw_df['AMOUNT_TO_RECOVER'] = auri_raw_df['AMOUNT_TO_RECOVER'].fillna(0)

    auri_raw_df.insert(2, 'AURI_SOURCE', auri_raw_df['AUDIT_KEY'].astype(str).str[:4])
    auri_raw_df['AURI_SOURCE'] = auri_raw_df['AURI_SOURCE'].replace({'CCIA': 'ECA audits'})
    auri_raw_df.loc[auri_raw_df['AUDIT_EXTENSION'] == 'Y', 'AURI_SOURCE'] = 'Extensions'
    auri_raw_df.loc[(auri_raw_df['AUDIT_EXTENSION'] == 'N') & (auri_raw_df['AURI_SOURCE'] != 'ECA audits'), 'AURI_SOURCE'] = 'CAS audits'

    return auri_raw_df

def auri_overview_df(auri_df: pd.DataFrame) -> pd.DataFrame:
    """Generate DataFrame for AURI Overview by Source with reordered columns."""
    sources =  ['CAS audits', 'ECA audits', 'Extensions', 'Total']
    data = []

    for source in sources:
        if source == 'Total':
            df_subset = auri_df
        else:
            df_subset = auri_df[auri_df['AURI_SOURCE'] == source]

        total = len(df_subset)
        processed = len(df_subset[df_subset['AURI_END_DATE'] != 'no'])
        pending = len(df_subset[df_subset['AURI_END_DATE'] == 'no'])
        pct_processed = (processed / total * 100) if total > 0 else 0
        pct_pending = (pending / total * 100) if total > 0 else 0

        data.append({
            'Source': source,
            'Audit results processed': processed,
            '% Audit results processed': pct_processed,
            'Audit results pending': pending,
            '% Audit results pending': pct_pending,
            'Total': total
        })

    return pd.DataFrame(data)

def tti_closed_projects_df(auri_df: pd.DataFrame, year: int) -> pd.DataFrame:
    """Generate DataFrame for TTI of Closed Projects with a Total row."""
    categories = [
        ('Negative with Recoveries*', lambda df: df[(df['AUDEX_TOTAL_COST_ADJUSTMENT'] < 0) & (df['AMOUNT_TO_RECOVER'] > 0)]),
        ('Negative without Recoveries', lambda df: df[(df['AUDEX_TOTAL_COST_ADJUSTMENT'] < 0) & (df['AMOUNT_TO_RECOVER'] == 0)]),
        ('Positive/Zero Adjustments', lambda df: df[df['AUDEX_TOTAL_COST_ADJUSTMENT'] >= 0])
    ]
    data = []

    base_df = auri_df[(auri_df['AURI_END_YEAR'] == year) & (auri_df['AURI_MODE'] == 'RO')]
    for category, filter_func in categories:
        df_subset = filter_func(base_df)
        total = len(df_subset)
        below_6m = len(df_subset[df_subset['TTI_AURI'] <= 180])
        above_6m = len(df_subset[df_subset['TTI_AURI'] > 180])
        pct_below = (below_6m / total * 100) if total > 0 else 0
        pct_above = (above_6m / total * 100) if total > 0 else 0

        data.append({
            'Adjustment Type': category,
            '0-6 months': below_6m,
            '% total (0-6 months)': pct_below,
            'above 6 months': above_6m,
            '% above 6 months': pct_above,
            'Total': total
        })

    df = pd.DataFrame(data)

    # Add Total row for Closed Projects
    total_row = pd.DataFrame([{
        'Adjustment Type': 'Closed Projects',
        '0-6 months': df['0-6 months'].sum(),
        '% total (0-6 months)': (df['0-6 months'].sum() / df['Total'].sum() * 100) if df['Total'].sum() > 0 else 0,
        'above 6 months': df['above 6 months'].sum(),
        '% above 6 months': (df['above 6 months'].sum() / df['Total'].sum() * 100) if df['Total'].sum() > 0 else 0,
        'Total': df['Total'].sum()
    }], index=['Closed_Total'])

    return pd.concat([total_row, df]).reset_index(drop=True)

def tti_open_projects_df(auri_df: pd.DataFrame, year: int) -> pd.DataFrame:
    """Generate DataFrame for TTI of Open Projects with a Total row."""
    categories = [
        ('Negative with REPA**', lambda df: df[df['AUDEX_TOTAL_COST_ADJUSTMENT'] < 0]),
        ('Positive/Zero Adjustments', lambda df: df[df['AUDEX_TOTAL_COST_ADJUSTMENT'] >= 0])
    ]
    data = []

    base_df = auri_df[(auri_df['AURI_END_YEAR'] == year) & (auri_df['AURI_MODE'] == 'REPA')]
    for category, filter_func in categories:
        df_subset = filter_func(base_df)
        total = len(df_subset)
        below_6m = len(df_subset[df_subset['TTI_AURI'] <= 180])
        above_6m = len(df_subset[df_subset['TTI_AURI'] > 180])
        pct_below = (below_6m / total * 100) if total > 0 else 0
        pct_above = (above_6m / total * 100) if total > 0 else 0

        data.append({
            'Adjustment Type': category,
            '0-6 months': below_6m,
            '% total (0-6 months)': pct_below,
            'above 6 months': above_6m,
            '% above 6 months': pct_above,
            'Total': total
        })

    df = pd.DataFrame(data)

    # Add Total row for On-going Projects
    total_row = pd.DataFrame([{
        'Adjustment Type': 'On-going Projects',
        '0-6 months': df['0-6 months'].sum(),
        '% total (0-6 months)': (df['0-6 months'].sum() / df['Total'].sum() * 100) if df['Total'].sum() > 0 else 0,
        'above 6 months': df['above 6 months'].sum(),
        '% above 6 months': (df['above 6 months'].sum() / df['Total'].sum() * 100) if df['Total'].sum() > 0 else 0,
        'Total': df['Total'].sum()
    }], index=['Open_Total'])

    return pd.concat([total_row, df]).reset_index(drop=True)


def negative_adjustments_df(auri_df: pd.DataFrame) -> pd.DataFrame:
    """Generate DataFrame for Negative Adjustments by Source with updated column names and order."""
    sources = ['CAS audits', 'ECA audits', 'Extensions', 'Total']
    data = []

    for source in sources:
        if source == 'Total':
            df_subset = auri_df[auri_df['AUDEX_TOTAL_COST_ADJUSTMENT'] <= -200]
        else:
            df_subset = auri_df[(auri_df['AURI_SOURCE'] == source) & (auri_df['AUDEX_TOTAL_COST_ADJUSTMENT'] <= -200)]

        total_count = len(df_subset)
        total_amount = df_subset['AUDEX_TOTAL_COST_ADJUSTMENT'].sum()
        processed = df_subset[df_subset['AURI_END_DATE'] != 'no']
        processed_count = len(processed)
        processed_amount = processed['AUDEX_TOTAL_COST_ADJUSTMENT'].sum()
        pending = df_subset[df_subset['AURI_END_DATE'] == 'no']
        pending_count = len(pending)
        pending_amount = pending['AUDEX_TOTAL_COST_ADJUSTMENT'].sum()

        data.append({
            'Source': source,
            'Processed No. of AURIs': processed_count,
            'Processed Adjustment Amount (AUDEX)': processed_amount,
            'Pending No. of AURIs': pending_count,
            'Pending Adjustment Amount (AUDEX)': pending_amount,
            'Total No. of AURIs': total_count,
            'Total Adjustment Amount (AUDEX)': total_amount,
        })

    # Create DataFrame with the desired column order
    df = pd.DataFrame(data)
    df = df[[
        'Source',
        'Processed No. of AURIs',
        'Processed Adjustment Amount (AUDEX)',
        'Pending No. of AURIs',
        'Pending Adjustment Amount (AUDEX)',
        'Total No. of AURIs',
        'Total Adjustment Amount (AUDEX)',
    ]]

    return df

def deviations_df(auri_df: pd.DataFrame, year: int) -> pd.DataFrame:
    """Generate DataFrame for Deviations."""
    deviations = auri_df[
        (auri_df['AURI_END_DATE'] != 'no') &
        (auri_df['AURI_END_YEAR'] == year) &
        (auri_df['AUDEX_TOTAL_COST_ADJUSTMENT'] <= -200) &
        (auri_df['DEVIATION'] == 'Y') &
        (auri_df['DEVIATION_AMOUNT'] < -1)
    ].copy()

    if deviations.empty:
        return pd.DataFrame(columns=[
            'Beneficiary', 'Project Number', 'Acronym', 'Count of AURI',
            'AURI Deviation Comment', 'Audex Total Cost Adjustment',
            'Deviation Amount', 'AURI Cost Adjustments'
        ])

    deviations = deviations[[
        'BENEFICIARY', 'PROJECT_NUMBER', 'ACRONYM', 'AURI_DEVIATION_COMMENT',
        'AUDEX_TOTAL_COST_ADJUSTMENT', 'DEVIATION_AMOUNT', 'AURI_COST_ADJUSTMENTS'
    ]]
    deviations['Count of AURI'] = 1
    deviations = deviations[[
        'BENEFICIARY', 'PROJECT_NUMBER', 'ACRONYM', 'Count of AURI',
        'AURI_DEVIATION_COMMENT', 'AUDEX_TOTAL_COST_ADJUSTMENT',
        'DEVIATION_AMOUNT', 'AURI_COST_ADJUSTMENTS'
    ]]

    total_row = pd.DataFrame([{
        'BENEFICIARY': 'TOTAL',
        'PROJECT_NUMBER': '',
        'ACRONYM': '',
        'Count of AURI': deviations['Count of AURI'].sum(),
        'AURI_DEVIATION_COMMENT': '',
        'AUDEX_TOTAL_COST_ADJUSTMENT': deviations['AUDEX_TOTAL_COST_ADJUSTMENT'].sum(),
        'DEVIATION_AMOUNT': deviations['DEVIATION_AMOUNT'].sum(),
        'AURI_COST_ADJUSTMENTS': deviations['AURI_COST_ADJUSTMENTS'].sum()
    }], index=['Total'])

    deviations = pd.concat([deviations, total_row])
    return deviations

import numpy as np
import pandas as pd
# ... (keep the other imports you already have)

def implementation_comparison_df(
    auri_df: pd.DataFrame,
    cutoff: pd.Timestamp,
    months_scope: list[int]
) -> pd.DataFrame:
    """
    CAS + ECA + Extensions rows, followed by
    'Total participation implemented'. All numeric columns are
    float/int or NaN (no strings).
    """
    last_date   = get_last_date_in_scope(cutoff, months_scope)
    current_lab = last_date.strftime("%b-%y")               # e.g. Mar-25
    prev_end    = pd.Timestamp(year=last_date.year - 1, month=12, day=31)
    prev_lab    = f"Dec-{str(prev_end.year)[-2:]}"          # e.g. Dec-24

    cur  = _snapshot(auri_df, last_date, current_lab)
    prev = _snapshot(auri_df, prev_end,  prev_lab)

    out = cur.merge(prev, on="Indicator", how="outer")

    # ── grand total ---------------------------------------------------
    num_cols = [c for c in out.columns if c != "Indicator" and not c.endswith("% cases")]
    pct_cols = [c for c in out.columns if c.endswith("% cases")]

    total = {"Indicator": "Total participation implemented"}
    for col in num_cols:
        total[col] = out[col].sum()

    # recompute % columns
    for lab in (current_lab, prev_lab):
        impl_sum = total[f"{lab} number of cases"]
        # estimate population = impl / (%/100) for each row, then sum
        pop_sum  = (
            out.apply(
                lambda r: (
                    r[f"{lab} number of cases"] * 100 / r[f"{lab} % cases"]
                    if pd.notna(r[f"{lab} % cases"]) and r[f"{lab} % cases"] > 0
                    else r[f"{lab} number of cases"]
                ),
                axis=1,
            ).sum()
        )
        total[f"{lab} % cases"] = (impl_sum / pop_sum * 100) if pop_sum else np.nan

    out = pd.concat([out, pd.DataFrame([total])], ignore_index=True)

    # make every non-stub column strictly numeric (NaN where coercion fails)
    for col in out.columns:
        if col != "Indicator":
            out[col] = pd.to_numeric(out[col], errors="coerce")

    return out



def apply_table_styling(gt: GT, table_type: str,  df_columns: list[str] | None = None, subtotal_background_color: str = LIGHT_BLUE) -> GT:
    """Apply consistent styling to great_tables objects based on table type."""
    if table_type == "auri_overview":
        first_col = ["Source"]
        other_cols = ["Audit results processed", "% Audit results processed", "Audit results pending", "% Audit results pending", "Total"]
    elif table_type == "tti_combined":
        first_col = ["Adjustment Type"]
        other_cols = ["0-6 months", "% total (0-6 months)", "above 6 months", "% above 6 months", "Total"]
    elif table_type == "negative_adj":
        first_col = ["Source"]
        other_cols = [
            "Processed No. of AURIs", "Processed Adjustment Amount (AUDEX)",
            "Pending No. of AURIs", "Pending Adjustment Amount (AUDEX)",
            "Total No. of AURIs", "Total Adjustment Amount (AUDEX)"
        ]
    elif table_type == "deviations":
         first_col = ["Beneficiary"]
         other_cols = ["Project Number", "Acronym", "Count of AURI", "AURI Deviation Comment", "Audex Total Cost Adjustment", "Deviation Amount", "AURI Cost Adjustments"]
    
    elif table_type == "participation_impl":
        first_col = ["Indicator"]
        # use the dataframe’s columns if caller passed them,
        # otherwise ask the GT object for its column names
        if df_columns is None:
            df_columns = list(gt.collect().columns)   # gt.collect() returns the underlying df
        other_cols = [c for c in df_columns if c not in first_col]
    else:
        raise ValueError(f"Unknown table type: {table_type}")

    gt = (
        gt
        .opt_table_font(font="Arial")
        .opt_table_outline(style="solid", width=OUTLINE_B, color=DARK_BLUE)

        # Row group styling
        .tab_style(
            style=[
                style.text(color=DARK_BLUE, weight="bold", font='Arial', size='small'),
                style.fill(color=LIGHT_BLUE),
                style.css(f"border: 1px solid {DARK_BLUE}; line-height:1.2; padding:5px;")
            ],
            locations=loc.row_groups()
        )

        # Column labels styling
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align="left", size='small'),
                style.css("min-width:50px; padding:5px; line-height:1.2")
            ],
            locations=loc.column_labels(columns=first_col)
        )
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align="right", size='small'),
                style.css("min-width:50px; padding:5px; line-height:1.2")
            ],
            locations=loc.column_labels(columns=other_cols)
        )

        # Stubhead styling
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align="center", size='small'),
                style.css("min-width:150px; padding:20px; line-height:1.2")
            ],
            locations=loc.stubhead()
        )

        # Body cell styling
        .tab_style(
            style=[
                style.borders(sides="all", color=DARK_BLUE, weight="1px"),
                style.text(align="left", size='small', font='Arial'),
                style.css("padding:5px")
            ],
            locations=loc.body(columns=first_col)
        )
        .tab_style(
            style=[
                style.borders(sides="all", color=DARK_BLUE, weight="1px"),
                style.text(align="right", size='small', font='Arial'),
                style.css("padding:5px")
            ],
            locations=loc.body(columns=other_cols)
        )

        # Stub cell styling
        .tab_style(
            style=[
                style.borders(sides="all", color=DARK_BLUE, weight="1px"),
                style.text(size='small', font='Arial'),
                style.css("padding:5px")
            ],
            locations=loc.stub()
        )

        # Table options
        .tab_options(
            table_body_border_bottom_color=DARK_BLUE,
            table_body_border_bottom_width="1px",
            table_border_right_color=DARK_BLUE,
            table_border_right_width="1px",
            table_border_left_color=DARK_BLUE,
            table_border_left_width="1px",
            table_border_top_color=DARK_BLUE,
            table_border_top_width="1px",
            column_labels_border_top_color=DARK_BLUE,
            column_labels_border_top_width="1px"
        )

        # Footer styling
        .tab_source_note("Source: Compass")
        .tab_source_note("Report: Audit Result Implementation")
        .tab_style(
            style=[
                style.text(size="small", font='Arial'),
                style.css("padding:5px; line-height:1.2")
            ],
            locations=loc.footer()
        )
    )

    # Apply subtotal background color to specific rows in tti_combined table
    if table_type == "tti_combined":
        gt = gt.tab_style(
            style=[
                style.fill(color=subtotal_background_color),
                style.text(weight="bold")
            ],
            locations=[
                loc.stub(rows=["Closed Projects", "On-going Projects", "Total"]),
                loc.body(rows=["Closed Projects", "On-going Projects", "Total"])
            ]
        )
    elif table_type == "negative_adj":
        # Apply styling to the Total row to match the column headers and spanners
        gt = gt.tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold")
            ],
            locations=[
                loc.stub(rows=["Total"]),
                loc.body(rows=["Total"])
            ]
        )
    elif table_type == "participation_impl":
        # Apply styling to the Total row to match the column headers and spanners
        gt = gt.tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold")
            ],
            locations=[
                loc.stub(rows=["Total participation implemented"]),
                loc.body(rows=["Total participation implemented"])
            ]
        )

    return gt

def generate_auri_tables(db_path: str, report_params: dict, cutoff: datetime, subtotal_background_color: str = SUB_TOTAL_BACKGROUND) -> tuple[GT, GT, GT, GT]:
    """Generate four AURI tables as great_tables objects."""

    months_scope = get_months_in_scope(cutoff)  # [1, 2, 3, 4]
    current_year = determine_epoch_year(cutoff)  # 2025

    auri_year  = current_year


    # Load and preprocess data
    auri_df = load_auri_data(db_path, cutoff, current_year, months_scope)

    # Table 1: AURI Overview
    df1 = auri_overview_df(auri_df)
    tbl1 = (
        GT(df1)
        .fmt_number(columns=['Audit results processed', 'Audit results pending', 'Total'], decimals=0)
        .fmt_percent(columns=['% Audit results processed', '% Audit results pending'], decimals=1, scale_values=False)
    )
    tbl1 = apply_table_styling(tbl1, table_type="auri_overview", subtotal_background_color=subtotal_background_color)
   
    # Combined Table: TTI Closed and Open Projects
    df2 = tti_closed_projects_df(auri_df, auri_year)
    df3 = tti_open_projects_df(auri_df, auri_year)
    df_combined = pd.concat([df2, df3]).reset_index(drop=True)

    # Add a grand total row at the bottom, summing only Closed Projects and On-going Projects
    subtotal_df = df_combined[df_combined['Adjustment Type'].isin(['Closed Projects', 'On-going Projects'])]
    total_row = pd.DataFrame([{
        'Adjustment Type': 'Total',
        '0-6 months': subtotal_df['0-6 months'].sum(),
        '% total (0-6 months)': (subtotal_df['0-6 months'].sum() / subtotal_df['Total'].sum() * 100) if subtotal_df['Total'].sum() > 0 else 0,
        'above 6 months': subtotal_df['above 6 months'].sum(),
        '% above 6 months': (subtotal_df['above 6 months'].sum() / subtotal_df['Total'].sum() * 100) if subtotal_df['Total'].sum() > 0 else 0,
        'Total': subtotal_df['Total'].sum()
    }], index=['Grand_Total'])

    df_combined = pd.concat([df_combined, total_row]).reset_index(drop=True)

    tbl2 = (
        GT(df_combined)
        .fmt_number(columns=['0-6 months', 'above 6 months', 'Total'], decimals=0)
        .fmt_percent(columns=['% total (0-6 months)', '% above 6 months'], decimals=1, scale_values=False)
    )
    tbl2 = apply_table_styling(tbl2, table_type="tti_combined", subtotal_background_color=subtotal_background_color)
    
    # Table 3: Negative Adjustments
    df4 = negative_adjustments_df(auri_df)
    tbl3 = (
        GT(df4)
        .fmt_number(columns=[
            'Total No. of AURIs', 'Processed No. of AURIs', 'Pending No. of AURIs'
        ], decimals=0)
        .fmt_number(columns=[
            'Total Adjustment Amount (AUDEX)', 'Processed Adjustment Amount (AUDEX)', 'Pending Adjustment Amount (AUDEX)'
        ], decimals=2, use_seps=True)
        # Reorder spanners to match desired order
        .tab_spanner(
            label="Audit results processed",
            columns=["Processed No. of AURIs", "Processed Adjustment Amount (AUDEX)"]
        )
        .tab_spanner(
            label="Audit results pending implementation",
            columns=["Pending No. of AURIs", "Pending Adjustment Amount (AUDEX)"]
        )
        .tab_spanner(
            label="Total Negative Adjustments",
            columns=["Total No. of AURIs", "Total Adjustment Amount (AUDEX)"]
        )
       # Rename columns for clarity
        .cols_label(
            Source="Source",
            **{
                "Processed No. of AURIs": "No. of AURIs",
                "Processed Adjustment Amount (AUDEX)": "Adjustment Amount (AUDEX)",
                "Pending No. of AURIs": "No. of AURIs",
                "Pending Adjustment Amount (AUDEX)": "Adjustment Amount (AUDEX)",
                "Total No. of AURIs": "No. of AURIs",
                "Total Adjustment Amount (AUDEX)": "Adjustment Amount (AUDEX)"
            }
        )
    )
    tbl3 = apply_table_styling(tbl3, table_type="negative_adj", subtotal_background_color=subtotal_background_color)

    # Table 4 (previously Table 5): Deviations
    df5 = deviations_df(auri_df, auri_year)
    tbl4 = (
        GT(df5)
        .fmt_number(columns=['Count of AURI'], decimals=0)
        .fmt_number(columns=['AUDEX_TOTAL_COST_ADJUSTMENT', 'DEVIATION_AMOUNT', 'AURI_COST_ADJUSTMENTS'], decimals=2, use_seps=True)
        .cols_label(
            BENEFICIARY="Beneficiary",
            PROJECT_NUMBER="Project Number",
            ACRONYM="Acronym",
            AURI_DEVIATION_COMMENT="AURI Deviation Comment",
            AUDEX_TOTAL_COST_ADJUSTMENT="Audex Total Cost Adjustment",
            DEVIATION_AMOUNT="Deviation Amount",
            AURI_COST_ADJUSTMENTS="AURI Cost Adjustments"
        )
    )
    tbl4 = apply_table_styling(tbl4, table_type="deviations", subtotal_background_color=subtotal_background_color)


    df_impl = implementation_comparison_df(auri_df, cutoff, months_scope)

    # labels for the two periods (already calculated inside implementation_comparison_df)
    current_lab = df_impl.columns[1].split(" ", 1)[0]   # e.g. "Mar-25"
    prev_lab    = df_impl.columns[4].split(" ", 1)[0]   # e.g. "Dec-24"

    tbl5 = (
        GT(df_impl, rowname_col="Indicator")
        # numeric formatting ------------------------------------------------
        .fmt_number(columns=[c for c in df_impl.columns if c.endswith("number of cases")],
                    decimals=0)
        .fmt_number(columns=[c for c in df_impl.columns if c.endswith("in €")],
                    decimals=2, use_seps=True)
        .fmt_percent(columns=[c for c in df_impl.columns if c.endswith("% cases")],
                    decimals=1, scale_values=False)
        # spanners for the two periods --------------------------------------
        .tab_spanner(label=current_lab,
                    columns=[c for c in df_impl.columns if c.startswith(current_lab)])
        .tab_spanner(label=prev_lab,
                    columns=[c for c in df_impl.columns if c.startswith(prev_lab)])
    )

    tbl5 = apply_table_styling(
        tbl5,
        table_type="participation_impl",
        df_columns=list(df_impl.columns),        # ← give the helper what it needs
        subtotal_background_color=SUB_TOTAL_BACKGROUND,
    )

    return tbl1, tbl2, tbl3, tbl4, tbl5

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__dealloc__'
Traceback (most recent call last):
  File "_zmq.py", line 169, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


In [61]:
epoch_year = determine_epoch_year(cutoff)
months_scope = get_months_in_scope(cutoff)
last_date = get_last_date_in_scope(cutoff, months_scope)  # 2025-03-31

df_auri = load_auri_data(DB_PATH, cutoff, epoch_year, months_scope)


df_comparison  = implementation_comparison_df(df_auri, cutoff, months_scope)

df_comparison 

DEBUG:root:Fetching latest data for table_alias: audit_result_implementation, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for audit_result_implementation: [('2025-05-23T08:05:46.082997', 13)]
DEBUG:root:Checking upload_id: 13, uploaded_at: 2025-05-23T08:05:46.082997
DEBUG:root:Fetched 1676 rows from audit_result_implementation with upload_id 13


,Indicator,Mar-25 number of cases,Mar-25 in €,Mar-25 % cases,Dec-24 number of cases,Dec-24 in €,Dec-24 % cases
0,CAS participation implemented,244,-4556378.05,84.722222,238,-4410096.13,86.861314
1,ECA participation implemented,4,-80056.56,100.0,4,-80056.56,100.0
2,Extensions participation implemented,98,-2289527.28,57.647059,96,-2244607.91,80.0
3,,,,,,,
4,Total participation implemented,346,-6925961.89,74.891775,338,-6734760.6,84.924623


In [ ]:
epoch_year

In [ ]:

df_auri_overview = auri_overview_df(df_auri)
df_auri_overview

In [ ]:
df_tticlosed_projects = tti_closed_projects_df(df_auri, epoch_year)
df_tticlosed_projects

In [ ]:
df_tti_open_projects = tti_open_projects_df(df_auri, epoch_year)
df_tti_open_projects

In [ ]:
df_neg_adj = negative_adjustments_df(df_auri)
df_neg_adj

In [ ]:
df_dev = deviations_df(df_auri, epoch_year)
df_dev

In [ ]:
months_scope = get_months_in_scope(cutoff)  # [1, 2, 3, 4]
current_year = determine_epoch_year(cutoff)  # 2025
last_date = get_last_date_in_scope(cutoff, months_scope)  # 2025-03-31

print(months_scope)
print(current_year)
print(last_date)

In [91]:
tbl1, tbl2, tbl3, tbl4, tbl5  = generate_auri_tables(DB_PATH, report_params, cutoff)

DEBUG:root:Fetching latest data for table_alias: audit_result_implementation, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for audit_result_implementation: [('2025-05-23T08:05:46.082997', 13)]
DEBUG:root:Checking upload_id: 13, uploaded_at: 2025-05-23T08:05:46.082997
DEBUG:root:Fetched 1676 rows from audit_result_implementation with upload_id 13


In [92]:
tbl5

GT(_tbl_data=                              Indicator  Mar-25 number of cases  Mar-25 in €  \
0         CAS participation implemented                     244  -4556378.05   
1         ECA participation implemented                       4    -80056.56   
2  Extensions participation implemented                      98  -2289527.28   
3       Total participation implemented                     346  -6925961.89   

   Mar-25 % cases  Dec-24 number of cases  Dec-24 in €  Dec-24 % cases  
0       84.722222                     238  -4410096.13       86.861314  
1      100.000000                       4    -80056.56      100.000000  
2       57.647059                      96  -2244607.91       80.000000  
3       74.891775                     338  -6734760.60       84.924623  , _body=<great_tables._gt_data.Body object at 0x000001611A7E4460>, _boxhead=Boxhead([ColInfo(var='Indicator', type=<ColInfoTypeEnum.stub: 2>, column_label='Indicator', column_align='left', column_width=None), ColInfo(var='Mar-25 number of cases', type=<ColInfoTypeEnum.default: 1>, column_label='Mar-25 number of cases', column_align='right', column_width=None), ColInfo(var='Mar-25 in €', type=<ColInfoTypeEnum.default: 1>, column_label='Mar-25 in €', column_align='right', column_width=None), ColInfo(var='Mar-25 % cases', type=<ColInfoTypeEnum.default: 1>, column_label='Mar-25 % cases', column_align='right', column_width=None), ColInfo(var='Dec-24 number of cases', type=<ColInfoTypeEnum.default: 1>, column_label='Dec-24 number of cases', column_align='right', column_width=None), ColInfo(var='Dec-24 in €', type=<ColInfoTypeEnum.default: 1>, column_label='Dec-24 in €', column_align='right', column_width=None), ColInfo(var='Dec-24 % cases', type=<ColInfoTypeEnum.default: 1>, column_label='Dec-24 % cases', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001611B879EB0>, _spanners=Spanners([SpannerInfo(spanner_id='Mar-25', spanner_level=0, spanner_label='Mar-25', spanner_units=None, spanner_pattern=None, vars=['Mar-25 number of cases', 'Mar-25 in €', 'Mar-25 % cases'], built=None), SpannerInfo(spanner_id='Dec-24', spanner_level=0, spanner_label='Dec-24', spanner_units=None, spanner_pattern=None, vars=['Dec-24 number of cases', 'Dec-24 in €', 'Dec-24 % cases'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Source: Compass', 'Report: Audit Result Implementation'], _footnotes=[], _styles=[StyleInfo(locname=LocRowGroups(rows=None), grpname={None}, colname=None, rownum=None, colnum=None, styles=[CellStyleText(color='#01244B', font='Arial', size='small', align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d6e6f4'), CellStyleCss(rule='border: 1px solid #01244B; line-height:1.2; padding:5px;')]), StyleInfo(locname=LocColumnLabels(columns=['Indicator']), grpname=None, colname='Indicator', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size='small', align='left', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='min-width:50px; padding:5px; line-height:1.2')]), StyleInfo(locname=LocColumnLabels(columns=['Mar-25 number of cases', 'Mar-25 in €', 'Mar-25 % cases', 'Dec-24 number of cases', 'Dec-24 in €', 'Dec-24 % cases']), grpname=None, colname='Mar-25 number of cases', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size='small', align='right', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='min-width:50px; padding:5px; line-height:1.2')]), StyleInfo(locname=LocColumnLabels(columns=['Mar-25 number of cases', 'Mar-25 in €', 'Mar-25 % cases', 'Dec-24 number of cases', 'Dec-24 in €', 'Dec-24 % cases']), grpname=None, colname='Mar-25 in €